In [127]:
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes,
    Graph,
    get_uri_name
)
from semantic_mpc_interface.utils import query_to_df
from semantic_mpc_interface.namespaces import BRICK, RDF, S223, A, HPFS, REF
from rdflib import URIRef, Literal
from buildingmotif import BuildingMOTIF, get_building_motif
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone
import json
# SELECT ONTOLOGY 
ontology = 'brick'
# base path
base_path = f'{ontology}'

In [128]:
run_inference = False

In [129]:
# switching out external references
boptest_graph = Graph()
boptest_graph.parse('post-processed-model.ttl', format = 'ttl')
remove_triples = []
for s, ref in boptest_graph.subject_objects(REF.hasExternalReference):
    for name in boptest_graph.objects(ref, REF.name):
        normal_type = name.split('_')[-2]
        normal_ext_ref = URIRef(str(s) + '_normal')
        boptest_graph.add((normal_ext_ref, REF.name, Literal(normal_type)))
        boptest_graph.add((normal_ext_ref, A, REF.NormalReference))
        boptest_graph.add((s, REF.hasExternalReference, normal_ext_ref))
        remove_triples.append((s, REF.hasExternalReference, ref))

#core zone not a zone for normal
boptest_graph.remove((URIRef('https://BESTESTAir.urn#flo_Cor'), A, BRICK.Zone))
boptest_graph.serialize('post-processed-model-normal.ttl', format = 'ttl')

<Graph identifier=N9a5f8324ab184c13af32d3e1d6db0fd5 (<class 'rdflib.graph.Graph'>)>

# Writing SHACL and Running Inference

In [130]:
# Please disregard excessive outputs (logging and warnings) from bmotif
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

Generating shapes "mints" our templates into an ontology 

The templates describe how certain things should be semantically represented in our graph. 
We can fill the templates to create these semantic representations.
However, using the SHACLHandler we can use the rules of the s223 or brick ontologies to invert the templates, and instead create validation rules looking for things represented like our templates.

The templates will be added as classes to a shacl graph, currently using the namespace HPFS (urn:hpflex/shapes), and we will infer what things in the graph match our templates. 

In [131]:
handler = SHACLHandler(ontology=ontology, template_dir = '/Users/lazlopaul/Desktop/223p/experiments/simbuild-semantic-25/templates')
# Generate shapes
handler.generate_shapes()
# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")


<Graph identifier=N3b1fbd1a031f44a6ae4b1c1c7ec48f00 (<class 'rdflib.graph.Graph'>)>

In [132]:
g = Graph()
g.parse('post-processed-model-normal.ttl', format = 'ttl')
og = clone.clone_graph(g)
# run if creating new inferred graph, take ~10 seconds
if run_inference:
    inferred_graph = handler.infer(g)
    inferred_graph.serialize('inferred_graph.ttl', format = 'ttl')
else:
    inferred_graph = Graph()
    inferred_graph.parse('inferred_graph.ttl', format = 'ttl')

Here we can see that many classes have been inferred based on our templates. 
All the zone temperatures will be labeled with the class HPFS:temp. Same for our zone temperatures.

We also infer some basic information, such as has-point. The semantic_model_builder currently uses these templates as the interface between the HPFS ontology and another ontology (i.e. brick or S223). This was done as an experimental approach for ontology alignment, but means that we must create some boilerplate templates to load data into objects.

In [133]:
i = 0
for triple in (inferred_graph - og):
    if ('Ref' in str(triple[0])) or ('has-point' in str(triple[1])):
        continue 
    else:
        print(triple)
    i += 1
    if i > 5:
        break

(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupCor_TZonHeaSet_normal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#normal-ref'))
(rdflib.term.URIRef('https://BESTESTAir.urn#heaPum_reaPPumDis_normal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#normal-ref'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonActCor_yDam_normal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#normal-ref'))
(rdflib.term.URIRef('https://BESTESTAir.urn#chi_reaPPumDis_normal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#normal-ref'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaAhu_V_flow_ret_normal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#normal-ref'))
(rdflib

In [134]:
list(inferred_graph[:A:HPFS['heaset']])

[rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupCor_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupNor_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupSou_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet')]

In [135]:
# Currently we load graphs from files for LoadModel, so must serialize this
inferred_graph.serialize(f"inferred_graph.ttl")

<Graph identifier=N0a37132c8c5d462682e6280d6de9df3a (<class 'rdflib.graph.Graph'>)>

# Demonstrating Loading Data 

LoadModel takes a filepath to a graph, a dictionary of mapping the objects you want to create, to the templates you want to create them with, the ontology used, and optionally a directory of templates.

site_info will contain dictionary of our desired templates. Each template may contain some other entities or properties. Entities and properties will be named arbitrarily based on the name of their templates. Properties are defined so that the have a value (a number, string, or external reference) and a unit. 

We want to get python objects representing each of our zones, so we will create objects based on our zone templates. These zone templates link to 3 properties for the temperature and setpoints. Each of these properties will have a value that points to the BOPTest external reference, and they will have a unit. NOTE: the units are currently incorrectly using the qudt namespace, rather than the unit namespace

In [136]:
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'zones': 'zone'}, 
                   ontology = ontology,
                   template_dir = 'templates')
site_info = loader.get_all_building_objects()

In [137]:
site_info

{'zones': [zone(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonCooSet_u')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')], unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonSou_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupSou

In [138]:
site_info['zones']

[zone(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonCooSet_u')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')], unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonSou_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupSou_TZonCooSet

In [139]:
for zone in site_info['zones']:
    print("ZONE NAME IS: ", zone.name)
    print("TEMP IS: ", zone.temp)
    print("TEMP DATA IS AT EXTERNAL REFERENCE: ", zone.temp.value)
    print("HEATING SETPOINT IS: ", zone.heaset)
    print("COOLING SEPTIONT IS: ", zone.cooset)
    print("Setpoint DATA is at External References: ", zone.heaset.value,', ', zone.cooset.value)
    print('-'* 20)

ZONE NAME IS:  https://BESTESTAir.urn#flo_Wes
TEMP IS:  Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')], unit='http://qudt.org/schema/qudt/K')
TEMP DATA IS AT EXTERNAL REFERENCE:  [Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')]
HEATING SETPOINT IS:  Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')], unit='http://qudt.org/schema/qudt/K')
COOLING SEPTIONT IS:  Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonCooSet_u')], unit='http://qudt.org/schema/qudt/K')
Setpoint DATA is at External References:  [Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')] ,  [Re

# Implementation in CDL

So, what goes into the CDL file? 

We have multiple options:

1. __Getting bindings using simple relations:__ 
    Say what points should be used for each controller, referencing the "minted ontology"/SHACL shapes from your templates (e.g. hpfs:temp, hpfs:heaset, hpfs:cooset.) We will have to create some predicates to do this (like hpfs:binds), but I think this may be the clearer.

2. __Representing controllers in semantic model:__
    Create templates representing the controller. Reference these templates in the CDL. Instantiate the templates to create s223 models of the controller for each zone and connect them to the properties in the model. We add the controller models to our graph. However, we will still have to do some querying probably to get the necessary data out for Normal. The expedient option may make more sense for now, but this was the option we landed on doing C2S

In [140]:
# Opt 1: just say what points should map to each input/output (and what template we should use for the query)
# ctrl:controller, ctrl:heaset_u, etc. would probably be written as <cdl_instance_name> in the modelica file. 
# there is some internal data (like occupancy) I think we don't really need, since it can be templatized as part of the controller in normal. I think we can handle it if we need to though
data = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix hpfs: <urn:hpflex/shapes#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

@prefix ctrl: <urn:mycontroller.urn#> .

ctrl:zone_ratcheting hpfs:controls hpfs:zone ;
    hpfs:uses ctrl:TZonHeaSetCur, ctrl:TZonCooSetCur,
        ctrl:TZonHeaSetCom, ctrl:TZonCooSetCom, ctrl:TZon .
    
ctrl:TZonHeaSetCur a s223:FunctionInput;
    hpfs:binds hpfs:heaset .

ctrl:TZonCooSetCur a s223:FunctionInput;
    hpfs:binds hpfs:cooset .

ctrl:TZon a s223:FunctionInput;
    hpfs:binds hpfs:temp .

ctrl:TZonCooSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:cooset .

ctrl:TZonHeaSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:heaset .

ctrl:occupancy hpfs:controls hpfs:zone ;
    hpfs:uses ctrl:TCooSet, ctrl:THeaSet .

ctrl:TCooSet a s223:FunctionInput;
    hpfs:binds hpfs:cooset .

ctrl:THeaSet a s223:FunctionInput;
    hpfs:binds hpfs:heaset .
"""

In [141]:
# Get our graph and the name of the template representing what we intend to control
ctrl_graph = Graph()
ctrl_graph.parse(data=data, format='turtle')
target_template_uri = list(ctrl_graph[:HPFS.controls:])[0][1]
target_template_name = get_uri_name(ctrl_graph, target_template_uri)
print('looking for template: ',target_template_name)

# Load the graph and get all the opjects matching this template, as before
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'control_target': target_template_name}, 
                   ontology = ontology,
                   template_dir = 'templates')

# runs a query for our controller template, which is the zone template. 
site_data = loader.get_all_building_objects()

# get targets of our controls
control_targets = site_data['control_target']

looking for template:  zone


In [142]:
# Look at controller model to get our controlers, their inputs, and their outputs. 
controller_dict = {}
for controller,_ in ctrl_graph[:HPFS.controls:]:
    controller_name = get_uri_name(ctrl_graph, controller)
    controller_dict[controller_name] = {}
    controller_dict[controller_name]['inputs'] = inputs = {}
    controller_dict[controller_name]['outputs'] = outputs = {}
    for point in ctrl_graph[controller:HPFS.uses:]:
        if (point, A, S223.FunctionInput) in ctrl_graph:
            input_name = get_uri_name(ctrl_graph, point)
            inputs[input_name] = get_uri_name(ctrl_graph, ctrl_graph.value(point, HPFS.binds))
        elif (point, A, S223.FunctionOutput) in ctrl_graph:
            output_name = get_uri_name(ctrl_graph, point)
            outputs[output_name] = get_uri_name(ctrl_graph, ctrl_graph.value(point, HPFS.binds))

In [143]:
controller_dict

{'zone_ratcheting': {'inputs': {'TZonHeaSetCur': 'heaset',
   'TZonCooSetCur': 'cooset',
   'TZon': 'temp'},
  'outputs': {'TZonHeaSetCom': 'heaset', 'TZonCooSetCom': 'cooset'}},
 'occupancy': {'inputs': {'TCooSet': 'cooset', 'THeaSet': 'heaset'},
  'outputs': {}}}

In [144]:
# Get the bindings from the semantic model for each input and output per controller
def get_normal_reference(ctrl_point_dict, control_targets = control_targets):
    input_bindings = {}
    for target in control_targets:
        for io, type in ctrl_point_dict.items():
            if hasattr(target, type):
                value_obj = getattr(target, type)
                for ref in value_obj.value:
                    if hasattr(ref, 'ref_type') and ref.ref_type == 'NormalReference':
                        input_bindings[io] = ref.name
    return input_bindings

In [145]:
get_normal_reference(controller_dict['zone_ratcheting']['outputs'])

{'TZonHeaSetCom': rdflib.term.Literal('TZonHeaSet', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),
 'TZonCooSetCom': rdflib.term.Literal('TZonCooSet', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))}

In [146]:
# Filling in the app.json template for normal with the semantic information from CDL and our model 

def query_formatter(equip_refs, input_refs):
    return {
        "type": "POINT_QUERY",

        "query": {
            "and": [
                {
                    "or": [
                        {
                            "field": {
                                "property": "equipRef",
                                "text": str(ref)
                            }
                        } for ref in sorted(equip_refs)
                    ]
                },
                {
                    "or": [
                        {
                            "field": {
                                "property": "class",
                                "text": str(ref_class)
                            }
                        } for ref_class in input_refs.values()
                    ]
                }
            ]
        },
        "label_attribute": "class",
        "groups": {
            "keys": ["equipRef"]
        },
        "query_mode": "POINT_QUERY_MODE_ADVANCED"
    }

# Fill in the queries using our IO 
def create_normal_queries(control_targets, inputs, outputs):
    # Extract unique equipRef values (e.g., "Eas", "Nor", "Sou", "Wes")
    # TODO: Add this as another normal reference to the model
    equip_refs = set()
    for target in control_targets:
        target_name = target.name.split('#')[-1]
        if '_' in target_name:
            equip_ref = target_name.split('_')[-1]
            equip_refs.add(equip_ref)
    
    input_refs = get_normal_reference(inputs)
    output_refs = get_normal_reference(outputs)
    
    query = query_formatter(equip_refs, input_refs)
    command = query_formatter(equip_refs, output_refs)
    
    return query, command 

# Create the flow edges from our io 
def create_flow_edges(controller_dict, query_id, command_id):
    flow_edges = []
    for controller in controller_dict.keys():
        inputs = controller_dict[controller]['inputs']
        input_refs = get_normal_reference(inputs)
        outputs = controller_dict[controller]['outputs']
        output_refs = get_normal_reference(outputs)
        id = controller_dict[controller]['id']
        for ctrl_name, ref_name in input_refs.items():
            flow_edges.append({
                "from": query_id,
                "to": id,
                "fromName": f"flow-source-handle-{ref_name}",
                "toName": f"flow-target-handle-{ctrl_name}"
            })
        for io in output_refs:
            flow_edges.append({
                "from": id,
                "to": command_id,
                "fromName": f"flow-source-handle-{ctrl_name}",
                "toName": f"flow-target-handle-{ref_name}"
            })
    return flow_edges

In [147]:
app_template = json.loads(open('app-template.json').read())

In [148]:
# Filling in the template 
query, command = create_normal_queries(control_targets, inputs, outputs)
flow_nodes = app_template['flowNodes']
for node in flow_nodes:
    if node['name'] == 'query':
        query_id = node['id']
        node['query'] = query
    elif node['name'] == 'command':
        command_id = node['id']
        node['command'] = command
    if node['name'] in controller_dict.keys():
        controller_dict[node['name']]['id'] = node['id']
        
new_flow_edges = create_flow_edges(controller_dict, query_id, command_id)

app_template['flowEdges'] = app_template['flowEdges'] + new_flow_edges


with open('app-completed.json', 'w') as f:
    json.dump(app_template, f, indent=2)

In [149]:
# O

# Old

In [150]:
# printout of bindings
bindings = list(ctrl_graph[:HPFS.binds:])
for target in control_targets:
    print(f"for Target: {target.name}")
    for binding in bindings:
        binding_name = binding[1].split('#')[-1]
        print(f"\nControl Point: {binding[0]}")
        print(f" - Binds to point {getattr(target, binding_name).name}")
        print(f' - Using a Reference Address {getattr(target, binding_name).value}')
    print('-'*20)



for Target: https://BESTESTAir.urn#flo_Wes

Control Point: urn:mycontroller.urn#TZonHeaSetCur
 - Binds to point https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet
 - Using a Reference Address [Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')]

Control Point: urn:mycontroller.urn#TZonHeaSetCom
 - Binds to point https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet
 - Using a Reference Address [Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')]

Control Point: urn:mycontroller.urn#THeaSet
 - Binds to point https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet
 - Using a Reference Address [Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')]

Control Point: urn:mycontroller.urn#TZonCooSetCur
 - Binds to point https://BESTESTAir.urn#hvac_ov